In [1]:
import pandas as pd
import bs4
import requests
import json
import re
import tqdm

import scipy.stats as st
import numpy as np

import bokeh
from bokeh.plotting import show as show_interactive
from bokeh.plotting import output_file, output_notebook
from bokeh.layouts import column, row
from bokeh.models import CustomJS, TextInput, LassoSelectTool, Select, MultiSelect, ColorBar, Legend, LegendItem
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Button, HTMLTemplateFormatter
from bokeh.events import SelectionGeometry
from bokeh.transform import linear_cmap, jitter
from bokeh.models import FactorRange


In [2]:
# As of 2020 https://www.ncbi.nlm.nih.gov/research/cog/
COG_dict = {
    "A" : "RNA processing and modification",
    "B" : "Chromatin structure and dynamics",
    "C" : "Energy production and conversion",
    "D" : "Cell cycle control, cell division, chromosome partitioning",
    "E" : "Amino acid transport and metabolism",
    "F" : "Nucleotide transport and metabolism",
    "G" : "Carbohydrate transport and metabolism",
    "H" : "Coenzyme transport and metabolism",
    "I" : "Lipid transport and metabolism",
    "J" : "Translation, ribosomal structure and biogenesis",
    "K" : "Transcription",
    "L" : "Replication, recombination, and repair",
    "M" : "Cell wall/membrane/envelope biogenesis",
    "N" : "Cell motility",
    "O" : "Posttranslational modification, protein turnover, chaperones",
    "P" : "Inorganic ion transport and metabolism",
    "Q" : "Secondary metabolites biosynthesis, transport and catabolism",
    "T" : "Signal transduction mechanisms",
    "U" : "Intracellular trafficking, secretion, and vesicular transport",
    "V" : "Defense mechanisms",
    "W" : "Extracellular structures",
    "X" : "Mobilome: prophages, transposons",
    "Y" : "Nuclear structure",
    "Z" : "Cytoskeleton",
    "R" : "General function prediction only",
    "S" : "Function unknown",
}

In [3]:
def term_count_dict_from_annotation_df(annot_df, term_column):
    
    column = annot_df[term_column].values
    
    funct_terms = []
    for entry in column:
        if entry != '-':
            if term_column == 'COG_category':
                # terms = [f'{e}: {COG_dict[e]}' for e in entry]
                terms = list(entry)
            else:
                terms = entry.split(',')
            for t in terms:
                funct_terms.append(t)

#     len(terms)
    
    term_count_dict = {}
    
    for t in funct_terms:
        count = term_count_dict.get(t, 0)
        count += 1
        term_count_dict[t] = count
        
    return term_count_dict

In [4]:
def enrichment_analysis(module, leiden_label_df, phases, background_annotation, term_column):
    
    module_ttids = leiden_label_df.loc[leiden_label_df[f'leiden_label_{phases}'] == module]['TTHERM_ID'].values
    
    module_annotation = background_annotation.loc[background_annotation['TTHERM_ID'].isin(module_ttids)]
    
    background_term_dict = term_count_dict_from_annotation_df(background_annotation, term_column)
    module_term_dict = term_count_dict_from_annotation_df(module_annotation, term_column)
    
    bs = []
    ps = []
    folds = []
    terms = []
    
    for t, module_count in module_term_dict.items():
        
        background_count = background_term_dict[t]
        module_size = len(module_annotation)
        background_size = len(background_annotation)
        
        standard_contingency_table = [
                                [module_count, background_count - module_count], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        # The -1 and +1 make this more conservative (see explanation from the DAVID database: 
        # https://david.ncifcrf.gov/helps/functional_annotation.html#geneenrich)
        conservative_contingency_table = [
                                [module_count - 1, background_count - module_count + 1], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        
        odds, p_standard = st.fisher_exact(standard_contingency_table, 'greater')
        odds, p_conservative = st.fisher_exact(conservative_contingency_table, 'greater')
        
        p_reasonable = np.mean([p_standard, p_conservative])
        
        bonferroni  = p_reasonable * len(module_term_dict)

        fold_enrichment = (module_count/module_size) / (background_count/background_size)

        if bonferroni <= 0.05:
            
            ps.append(p_reasonable)
            bs.append(bonferroni)
            folds.append(fold_enrichment)
            terms.append(t)
            
#         else:
#             ps.append('')
#             bs.append('')
#             folds.append('')
#             terms.append('')
            
    return ps, bs, folds, terms
            
            

In [5]:
def get_GO_info(go_term):
    
    r = requests.get(f'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{go_term}/complete', 'html5lib')
    
    go_info = json.loads(r.text)
    
    name = go_info['results'][0]['name']
    
    definition = go_info['results'][0]['definition']['text']
    
    obsolete = go_info['results'][0]['isObsolete']
    
    return name, definition, obsolete

In [6]:
def get_KEGG_info(term):
    try:
#         dfs = pd.read_html(f'https://www.genome.jp/entry/{term}')
        response = requests.get(f'https://rest.kegg.jp/get/{term}')
    except:
        return "NA"

    text = response.text
    name_string = re.search(r'NAME\s*[\w \-]*', response.text).group()
    name = re.split(r'\s\s+', name_string)[1]
#     data = dfs[3]
#     entry = data.loc[data[0] == 'Entry'][1].values[0]
#     name = data.loc[data[0] == 'Name'][1].values[0]
    
    return name

In [7]:
def get_EC_info(term):
    
    r = requests.get(f"https://www.ebi.ac.uk/enzymeportal/rest/enzymes/{term}")
    try:
        name = r.json()['enzymeName']
    except:
        name = 'NA'
    
    return name

In [8]:
def write_enrichment(lldf, phases, out_prefix, term_column):
    
    with open(f'../enrichment/{out_prefix}_enrichment.txt', 'w') as fl:
    
        for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):

            # print(f'Module {m}')
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, complete_annot, term_column)

            fl.write(f'Module {m}\n')
            for b, fold, t in zip(bs, folds, terms):
                
                if term_column == 'COG_category':
                    
                    definition = COG_dict[t]
                    t = f'{t}: {definition}'
                
                elif term_column == 'GOs':
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
                    else:
                        t = f'{name.capitalize()} ({t}): {definition}'
                        
                elif term_column.split('_')[0] == 'KEGG':
                    name = get_KEGG_info(t)
                    
                    t = f'{t}: {name}'
                    
                elif term_column == 'EC':
                    
                    name = get_EC_info(t)
                    t = f'{t}: {name}'
                    
                        
                # print(f"{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n")

                fl.write(f'{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n')

In [9]:


response = requests.get('https://rest.kegg.jp/get/ko:K19607')

In [10]:
test = response.text.split('\n')

In [11]:
name = re.search(r'NAME\s*[\w \-]*', response.text).group()

In [12]:
re.split(r'\s\s+', name)

['NAME', 'X-linked retinitis pigmentosa GTPase regulator']

In [13]:
name

'NAME        X-linked retinitis pigmentosa GTPase regulator'

In [14]:
test[:10]

['ENTRY       K19607                      KO',
 'SYMBOL      RPGR',
 'NAME        X-linked retinitis pigmentosa GTPase regulator',
 'DISEASE     H00481  Cone-rod dystrophy and cone dystrophy',
 '            H00527  Retinitis pigmentosa',
 'BRITE       KEGG Orthology (KO) [BR:ko00001]',
 '             09180 Brite Hierarchies',
 '              09183 Protein families: signaling and cellular processes',
 '               03037 Cilium and associated proteins',
 '                K19607  RPGR; X-linked retinitis pigmentosa GTPase regulator']

In [15]:
r = requests.get("https://www.ebi.ac.uk/enzymeportal/rest/enzymes/2.7.11.1")
r.json()['enzymeName']

'Non-specific serine/threonine protein kinase'

In [16]:
def get_enrichment_df(lldf, phases, background_annotation, term_columns=['COG_category', 'GOs', 'KEGG_ko', 'EC'], outfile=None):
    
    module_dfs = []
    
    for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):
        
        term_dfs = []

        for tc in term_columns:
        
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, background_annotation, tc)

            # fl.write(f'Module {m}\n')
            
            info = []

            if tc == 'GOs':

                for t in terms:
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        info.append(f'{name.capitalize()}: {definition} (obsolete)')
                    else:
                        info.append(f'{name.capitalize()}: {definition}')
                        
            elif tc == 'COG_category':
                for t in terms:
                    info.append(COG_dict[t])
                                
            elif tc == 'KEGG_ko':
                for t in terms:
                    info.append(get_KEGG_info(t))
                    
            elif tc == 'EC':
                for t in terms:
                    info.append(get_EC_info(t))
                    
            term_df = pd.DataFrame({'module': [m]*len(terms),
                                    'term': terms,
                                    'info': info,
                                    'fold_change': folds,
                                    'bonferroni': bs})
            
            term_dfs.append(term_df)
            
        module_df = pd.concat(term_dfs)
        
        module_dfs.append(module_df)
        
    all_enrichment_df = pd.concat(module_dfs)
    
    if outfile:
        all_enrichment_df.to_csv(outfile, index=False)
    
    return all_enrichment_df
        
        
                        
            
                    
            
        
#         for b, fold, t in zip(bs, folds, terms):

#             if term_column == 'GOs':
#                 name, definition, obsolete = get_GO_info(t)
#                 if obsolete:
#                     t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
#                 else:
#                     t = f'{name.capitalize()} ({t}): {definition}'

#             if term_column.split('_')[0] == 'KEGG':
#                 dfs = pd.read_html(f'https://www.genome.jp/entry/{t}')

#                 data = dfs[4]
#                 entry = data.loc[data[0] == 'Entry'][1].values[0]
#                 name = data.loc[data[0] == 'Name'][1].values[0]

#                 t = f'{t}: {name}'

In [17]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')

In [18]:
lldf_nn3 = pd.read_csv('../embedding/mcl_rcl_res400_label_df_round_1_rearranged.csv')

In [19]:
lldf_nn3['leiden_label_full'].unique()

array([560., 111., 374., 548., 136., 240., 511., 401., 360., 603., 695.,
       687., 725., 755., 497., 306., 218., 684., 232.,  55., 455., 772.,
       787., 606.,  82., 373., 366., 661., 223., 166., 581., 142., 154.,
       364., 209., 248., 336., 294., 281., 650., 609.,   0., 666., 419.,
       692., 728., 565., 194., 309., 553., 405., 408., 465.,  25., 427.,
       651., 793., 487., 496., 739., 414., 586., 507.,  53., 208.,  89.,
       489., 707., 264., 483., 721., 202., 398., 190., 363., 224., 315.,
        37., 574., 520., 113., 474.,  95., 756., 665., 214., 456., 681.,
       275., 103., 699., 185., 482., 450., 749.,   6., 311.,  56., 591.,
       235.,  73., 671., 186., 127., 221., 179., 610., 342.,  83., 249.,
       771., 277.,  66.,  28., 673., 141.,   5., 258., 421., 600., 800.,
       719., 752.,  49., 759., 219., 134., 806., 512., 161., 261., 274.,
       636., 121., 502., 430., 594., 649., 658., 361., 698., 297., 104.,
       271., 593., 225., 130.,  17., 382., 660., 17

In [20]:
%pdb

Automatic pdb calling has been turned ON


In [21]:
enrich_full_nn3 = get_enrichment_df(lldf_nn3, 'full', complete_annotation, outfile='./test_nn3_full_enrichment.csv')

  0%|          | 0/404 [00:00<?, ?it/s]

100%|██████████| 404/404 [04:17<00:00,  1.57it/s]


In [22]:
enrich_full_nn3 = pd.read_csv('./test_nn3_full_enrichment.csv')

In [23]:
enrich_full_nn3.head()

,module,term,info,fold_change,bonferroni
0,6.0,U,"Intracellular trafficking, secretion, and vesi...",16.349938,0.000103
1,19.0,2.7.11.1,Non-specific serine/threonine protein kinase,13.373058,0.001371
2,27.0,O,"Posttranslational modification, protein turnov...",6.315055,0.043028
3,52.0,C,Energy production and conversion,26.084106,0.009340
4,55.0,A,RNA processing and modification,20.402486,0.001699


In [24]:
enrich_full_nn3['module'].unique()

array([  6.,  19.,  27.,  52.,  55.,  56.,  70.,  71.,  73.,  84.,  89.,
        90.,  94., 108., 111., 116., 117., 134., 145., 186., 189., 214.,
       217., 218., 220., 223., 229., 230., 233., 245., 250., 261., 270.,
       272., 277., 280., 285., 287., 294., 308., 309., 313., 322., 331.,
       334., 350., 358., 360., 370., 376., 379., 384., 385., 401., 402.,
       407., 413., 426., 455., 458., 483., 493., 495., 496., 497., 498.,
       548., 549., 552., 555., 561., 567., 575., 578., 581., 587., 598.,
       600., 608., 609., 611., 613., 616., 622., 637., 639., 644., 650.,
       652., 655., 665., 666., 668., 673., 693., 699., 714., 716., 723.,
       731., 738., 751., 752., 755., 756., 757., 759., 771., 772., 777.,
       778., 780., 788., 806.])

In [25]:
lldf_nn3['leiden_label_full'].unique()

array([560., 111., 374., 548., 136., 240., 511., 401., 360., 603., 695.,
       687., 725., 755., 497., 306., 218., 684., 232.,  55., 455., 772.,
       787., 606.,  82., 373., 366., 661., 223., 166., 581., 142., 154.,
       364., 209., 248., 336., 294., 281., 650., 609.,   0., 666., 419.,
       692., 728., 565., 194., 309., 553., 405., 408., 465.,  25., 427.,
       651., 793., 487., 496., 739., 414., 586., 507.,  53., 208.,  89.,
       489., 707., 264., 483., 721., 202., 398., 190., 363., 224., 315.,
        37., 574., 520., 113., 474.,  95., 756., 665., 214., 456., 681.,
       275., 103., 699., 185., 482., 450., 749.,   6., 311.,  56., 591.,
       235.,  73., 671., 186., 127., 221., 179., 610., 342.,  83., 249.,
       771., 277.,  66.,  28., 673., 141.,   5., 258., 421., 600., 800.,
       719., 752.,  49., 759., 219., 134., 806., 512., 161., 261., 274.,
       636., 121., 502., 430., 594., 649., 658., 361., 698., 297., 104.,
       271., 593., 225., 130.,  17., 382., 660., 17

In [26]:
lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == 'TTHERM_01386050']

,Unnamed: 0,TTHERM_ID,leiden_label_full
11979,12635,TTHERM_01386050,217.0


end here. Return to embedding notebook!